In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler

Using TensorFlow backend.


In [2]:
# define network parameters
max_features = 128
maxlen = 128

In [3]:
#read csv
dict_label = {'Usable directly numeric':0, 'Usable with extraction':1, 'Usable with Extration': 1, 'Usable with extraction ':1, 'Usable directly categorical':2, 'Unusable':3, 'Context_specific':4, 'Usable directly categorical ':2}
data = pd.read_csv('data_for_ML_num.csv')
data['y_act'] = [dict_label[i] for i in data['y_act']]
y = data.loc[:,['y_act']]
data_LSTM = pd.concat([data['Attribute_name'], data['sample_1'], data['sample_2'], data['sample_3'], data['sample_4'], data['sample_5']], axis =1)

# X_train, X_test,y_train,y_test = train_test_split(data,y, test_size=0.2)

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,5,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data = data.rename(columns={'Num of nans': 'Num_of_nans', 'num of dist_val': 'num_of_dist_val'})

data['Num_of_nans'] = [float(data['Num_of_nans'][i])/float(data['Total_val'][i]) for i in data.index]
data['num_of_dist_val'] = [float(data['num_of_dist_val'][i])/float(data['Total_val'][i]) for i in data.index]

data1 = data[['Num_of_nans', 'max_val', 'mean', 'min_val', 'num_of_dist_val','std_dev','castability','extractability', 'len_val']]
data1 = data1.fillna(0)
print('hghj')
data1 = data1.rename(columns={'mean': 'scaled_mean', 'min_val': 'scaled_min_val', 'max_val': 'scaled_max_val','std_dev': 'scaled_std_dev'})
data1.loc[data1['scaled_min_val'] > 10000, 'scaled_min_val'] = 10000
data1.loc[data1['scaled_min_val'] < -10000, 'scaled_min_val'] = -10000
data1.loc[data1['scaled_max_val'] > 10000, 'scaled_max_val'] = 10000
data1.loc[data1['scaled_max_val'] < -10000, 'scaled_max_val'] = -10000
data1.loc[data1['scaled_mean'] > 10000, 'scaled_mean'] = 10000
data1.loc[data1['scaled_mean'] < -10000, 'scaled_mean'] = -10000
data1.loc[data1['scaled_std_dev'] > 10000, 'scaled_std_dev'] = 10000
data1.loc[data1['scaled_std_dev'] < -10000, 'scaled_std_dev'] = -10000
column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev']
x = data1[column_names_to_normalize].values
x = np.nan_to_num(x)
x_scaled = StandardScaler().fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data1.index)
data1[column_names_to_normalize] = df_temp

data1.Num_of_nans = data1.Num_of_nans.astype(float)
data1.num_of_dist_val = data1.num_of_dist_val.astype(float)
data1.castability = data1.castability.astype(float)
data1.extractability = data1.extractability.astype(float)
y.y_act = y.y_act.astype(float)

# import enchant
# data1.to_csv('before.csv')
# f = open('current.txt','w')
# d = enchant.Dict("en_US")

# for i in data.index:
#     ival = data.at[i,'Attribute_name']
#     if ival != 'id' and d.check(ival):
#         print >> f,ival
#         print >> f,y.at[i,'y_act']
#         data1.at[i,'dictionary_item'] = 1
#     else:
#         data1.at[i,'dictionary_item'] = 0

# data1.to_csv('after.csv')
# f.close()
# from sklearn.feature_extraction.text import CountVectorizer


# arr = data['Attribute_name'].values
# data = data.fillna(0)
# arr1 = data['sample_1'].values
# arr1 = [str(x) for x in arr1]
# arr2 = data['sample_2'].values
# arr2 = [str(x) for x in arr2]
# arr3 = data['sample_3'].values
# arr3 = [str(x) for x in arr3]

# # print(arr)
# # print(arr1)
# vectorizer = CountVectorizer(ngram_range=(3,3),analyzer='char')
# X = vectorizer.fit_transform(arr)
# X1 = vectorizer.fit_transform(arr1)
# X2 = vectorizer.fit_transform(arr2)
# X3 = vectorizer.fit_transform(arr3)

# print(len(vectorizer.get_feature_names()))

# data1.to_csv('before.csv')
# tempdf = pd.DataFrame(X.toarray())
# tempdf1 = pd.DataFrame(X1.toarray())
# tempdf2 = pd.DataFrame(X2.toarray())
# tempdf3 = pd.DataFrame(X3.toarray())

# data2 = pd.concat([data1,tempdf], axis=1, sort=False)
# data2.to_csv('after.csv')


print('hghj')
X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=100)

key_name = data['Attribute_name']
atr_train,atr_test = train_test_split(key_name, test_size=0.2,random_state=100)
# print(atr_train)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
atr_train.reset_index(inplace=True,drop=True)
atr_test.reset_index(inplace=True,drop=True)

list_sentences_train = atr_train.values
list_sentences_test = atr_test.values


structured_data_train = X_train 
structured_data_test = X_test


# structured_data_train = X_train.loc[:,['Num_of_nans', 'scaled_max_val', 'scaled_mean', 'scaled_min_val', 'num_of_dist_val','scaled_std_dev','castability','extractability', 'len_val']]
# structured_data_test = X_test.loc[:,['Num_of_nans', 'scaled_max_val', 'scaled_mean', 'scaled_min_val', 'num_of_dist_val','scaled_std_dev','castability','extractability', 'len_val']]

# structured_data_train = structured_data_train.reset_index(drop=True)
# structured_data_test = structured_data_test.reset_index(drop=True)


structured_input_train = []
for i in range(len(structured_data_train)):
#     print(structured_data_train.loc[i])
#     print(list(structured_data_train.loc[i]))
    structured_input_train.append(list(structured_data_train.loc[i]))
structured_input_train = np.array(structured_input_train).reshape(len(structured_data_train),len(structured_data_train.keys()))    

structured_input_test = []
for i in range(len(structured_data_train)):
    structured_input_test.append(list(structured_data_train.loc[i]))
structured_input_test = np.array(structured_input_test).reshape(len(structured_data_train),len(structured_data_train.keys()))    


hghj
hghj


In [5]:
key_name = data['sample_1']
samp1_train,samp1_test = train_test_split(key_name, test_size=0.2,random_state=100)
samp1_train.reset_index(inplace=True,drop=True)
samp1_test.reset_index(inplace=True,drop=True)


key_name = data['sample_2']
samp2_train,samp2_test = train_test_split(key_name, test_size=0.2,random_state=100)
samp2_train.reset_index(inplace=True,drop=True)
samp2_test.reset_index(inplace=True,drop=True)

In [6]:

list_sentences_train = atr_train.values
list_sentences_test = atr_test.values

# X_train.sample_1 = X_train.sample_1.astype(str)
# X_test.sample_1 = X_test.sample_1.astype(str)

list_sentences_train1 = samp1_train.values
list_sentences_test1 = samp1_test.values

print(list_sentences_train1)

# X_train.sample_2 = X_train.sample_2.astype(str)
# X_test.sample_2 = X_test.sample_2.astype(str)

list_sentences_train2 = samp2_train.values
list_sentences_test2 = samp2_test.values

print(list_sentences_train2)

for i in range(len(list_sentences_train)):
    list_sentences_train[i] = str(list_sentences_train[i])
    list_sentences_train1[i] = str(list_sentences_train1[i])
    list_sentences_train2[i] = str(list_sentences_train2[i])
    
for i in range(len(list_sentences_test)):
    list_sentences_test[i] = str(list_sentences_test[i])
    list_sentences_test1[i] = str(list_sentences_test1[i])
    list_sentences_test2[i] = str(list_sentences_test2[i])    


['#NULL!' '0' '268' ... 'Hate' '-0.101' '2']
['0' '78' '110' ... nan '0.171' '5']


In [7]:
tokenizer = keras_text.Tokenizer(char_level = True)
tokenizer.fit_on_texts(list(list_sentences_train))
# train data
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=maxlen)
# test data
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = keras_seq.pad_sequences(list_tokenized_test, maxlen=maxlen)


tokenizer1 = keras_text.Tokenizer(char_level = True)
tokenizer1.fit_on_texts(list(list_sentences_train1))
# train data
list_tokenized_train1 = tokenizer.texts_to_sequences(list_sentences_train1)
X_t1 = keras_seq.pad_sequences(list_tokenized_train1, maxlen=maxlen)
# test data
list_tokenized_test1 = tokenizer.texts_to_sequences(list_sentences_test1)
X_te1 = keras_seq.pad_sequences(list_tokenized_test1, maxlen=maxlen)


tokenizer2 = keras_text.Tokenizer(char_level = True)
tokenizer2.fit_on_texts(list(list_sentences_train1))
# train data
list_tokenized_train2 = tokenizer.texts_to_sequences(list_sentences_train2)
X_t2 = keras_seq.pad_sequences(list_tokenized_train2, maxlen=maxlen)
# test data
list_tokenized_test2 = tokenizer.texts_to_sequences(list_sentences_test2)
X_te2 = keras_seq.pad_sequences(list_tokenized_test2, maxlen=maxlen)

# print(tokenizer)

In [10]:

bestone = load_model('best_weights.h5')
print('---')
loss, acc = bestone.evaluate([X_t,X_t1,structured_data_train],to_categorical(y_train),verbose=1)
print(loss, acc)
loss, acc = bestone.evaluate([X_te,X_te1,structured_data_test],to_categorical(y_test),verbose=1)
print(loss, acc)


---
7400/7400 [==============================] - 1s 96us/step
(0.050643856878568594, 0.9816216216216216)
1851/1851 [==============================] - 0s 68us/step
(0.5080923764513094, 0.8995137766591282)
